In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [9]:
def load_data():
  # Load MNIST & NIST dataset
  digits_data_train = pd.read_csv("sample_data/mnist_train.csv")
  digits_data_test = pd.read_csv("sample_data/mnist_test.csv")
  letters_dataset = pd.read_csv("sample_data/A_Z Handwritten Data.csv")
  digits_data = pd.concat([digits_data_train, digits_data_test], ignore_index=True)
  # Rename correct label column name to label
  digits_data.rename(columns={'0':'label'}, inplace=True)
  letters_dataset.rename(columns={'0':'label'}, inplace=True)

  # Select 1000 samples from each class
  digits_data = digits_data.groupby('label').head(1000)
  letters_dataset = letters_dataset.groupby('label').head(1000)

  # Split data into X and Y for each type
  Y1 = digits_data['label']
  X1 = digits_data.drop('label', axis=1)
  Y2 = letters_dataset["label"]
  X2 = letters_dataset.drop("label", axis=1)

  # Split data into train and test set
  x_train1, x_test1, y_train1, y_test1 = train_test_split(X1, Y1, train_size=0.9)
  x_train2, x_test2, y_train2, y_test2 = train_test_split(X2, Y2, train_size=0.9)

  # Convert into numpy array to ease preprocessing
  x_train1 = x_train1.to_numpy()
  x_test1 = x_test1.to_numpy()
  x_train2 = x_train2.to_numpy()
  x_test2 = x_test2.to_numpy()

  # Add ten to match number of classes for one-hot
  y_train2 = y_train2 + 10
  y_test2 = y_test2 + 10

  # Convert Y into one-hot vectors
  y_train1 = to_categorical(y_train1, num_classes=36)
  y_test1 = to_categorical(y_test1, num_classes=36)
  y_train2 = to_categorical(y_train2, num_classes=36)
  y_test2 = to_categorical(y_test2, num_classes=36)

  # Reshape Xs into CNN input dimension
  x_train1 = x_train1.reshape(x_train1.shape[0], 28, 28, 1)
  x_test1 = x_test1.reshape(x_test1.shape[0], 28, 28, 1)
  x_train2 = x_train2.reshape(x_train2.shape[0], 28, 28, 1)
  x_test2 = x_test2.reshape(x_test2.shape[0], 28, 28, 1)

  # Combine each X and Y from each dataset
  x_train = np.concatenate((x_train1, x_train2), axis=0)
  x_test = np.concatenate((x_test1, x_test2), axis=0)
  y_train = np.concatenate((y_train1, y_train2), axis=0)
  y_test = np.concatenate((y_test1, y_test2), axis=0)

  return x_train, x_test, y_train, y_test

def preprocessing(x_train, x_test):
  x_train = x_train / 255
  x_test = x_test / 255
  return x_train, x_test

In [10]:
x_train, x_test, y_train, y_test = load_data()
(x_train, x_test) = preprocessing(x_train, x_test)
# Initialize Keras image generator
generator = ImageDataGenerator(rotation_range=10, zoom_range=0.1, width_shift_range=0.1, height_shift_range=0.1)

# CNN Model
model = Sequential([
    Conv2D(filters=32, kernel_size=3, input_shape=(28, 28, 1), activation="relu"),
    BatchNormalization(),
    Conv2D(filters=32, kernel_size=3, activation="relu"),
    BatchNormalization(),
    Conv2D(filters=32, kernel_size=5, strides=2, padding="same", activation="relu"),
    BatchNormalization(),
    Dropout(0.4),
    Conv2D(filters=64, kernel_size=3, activation="relu"),
    BatchNormalization(),
    Conv2D(filters=64, kernel_size=3, activation="relu"),
    BatchNormalization(),
    Conv2D(filters=64, kernel_size=5, strides=2, padding="same", activation="relu"),
    BatchNormalization(),
    Dropout(0.4),
    Conv2D(128, kernel_size=4, activation="relu"),
    BatchNormalization(),
    Flatten(),
    Dropout(0.4),
    Dense(36, activation="softmax")
])

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

history = model.fit(generator.flow(x_train, y_train, batch_size=64), epochs=30, steps_per_epoch=x_train.shape[0] // 64, validation_data=(x_test, y_test))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


506/506 ━━━━━━━━━━━━━━━━━━━━ 164s 299ms/step - accuracy: 0.3876 - loss: 2.3860 - val_accuracy: 0.4950 - val_loss: 1.7814
Epoch 2/30
  1/506 ━━━━━━━━━━━━━━━━━━━━ 1:46 210ms/step - accuracy: 0.8750 - loss: 0.5450

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


506/506 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8750 - loss: 0.5450 - val_accuracy: 0.5061 - val_loss: 1.7075
Epoch 3/30
506/506 ━━━━━━━━━━━━━━━━━━━━ 149s 295ms/step - accuracy: 0.8230 - loss: 0.5688 - val_accuracy: 0.9253 - val_loss: 0.1986
Epoch 4/30
506/506 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.8125 - loss: 0.7359 - val_accuracy: 0.9258 - val_loss: 0.1974
Epoch 5/30
506/506 ━━━━━━━━━━━━━━━━━━━━ 152s 300ms/step - accuracy: 0.8661 - loss: 0.4147 - val_accuracy: 0.9356 - val_loss: 0.1922
Epoch 6/30
506/506 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9062 - loss: 0.3371 - val_accuracy: 0.9347 - val_loss: 0.1924
Epoch 7/30
506/506 ━━━━━━━━━━━━━━━━━━━━ 196s 297ms/step - accuracy: 0.8849 - loss: 0.3429 - val_accuracy: 0.9350 - val_loss: 0.1896
Epoch 8/30
506/506 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9219 - loss: 0.1744 - val_accuracy: 0.9369 - val_loss: 0.1873
Epoch 9/30
506/506 ━━━━━━━━━━━━━━━━━━━━ 198s 294ms/step - accuracy: 0.8956 - loss: 0.3109 - val_accurac

In [11]:
model.save('handwriting_model.h5')